In [ ]:
import math
from PIL import Image, ImageDraw
import requests
import xml.dom.minidom as dom

ROUGE = (255,0,0)
GRIS = (120,120,120)
BLEU = (0,0,255)
BLANC = (255,255,255)

R_TERRE = 6_378_137
CIRCONF = int(2 * math.pi * R_TERRE)

# Cartographie WMTS

*(Web Map Tile Service)*

C'est un protocole qui permet d'échanger tout ou partie d'images tuilées multi-résolution de la surface terrestre.

<img width="1024" src="img/Rendu-niveaux_carte.png" alt="3 premiers niveaux de résolution">

Habituellement, chaque tuile a une définition de 256x256 et les niveaux vont de 0 à 18.

Sommaire:

- [Repère WMTS](#Repère-WMTS)
- [Niveaux et tuiles WMTS](#Niveaux-de-résolution-et-tuiles-WMTS)
- [Géoservices](#Géoservice)
    - [Cartes IGN](#Cartes-IGN)
    - [Géocodage](#Géocodage)

## Repère WMTS

La planisphère est projetée (Web Mercator) sur un carré dont le côté correspond à la circonférence de la Terre. L'origine se trouve dans le coin supérieur gauche, l'axe des abscisses est orienté vers la droite, l'axe des ordonnées est orienté vers le bas.

In [ ]:
repere = Image.open('img/terre.jpg')

draw = ImageDraw.Draw(repere)
draw.line([(0,0), (512,0)], ROUGE, 5)
draw.line([(0,0), (0,512)], ROUGE, 5)
draw.line([(256,0), (256,512)], GRIS, 1)
draw.line([(0,256), (512, 256)], GRIS, 1)
draw.ellipse([(-2,-2), (2,2)], ROUGE)
draw.text((2,2), '(0,0)', ROUGE)
draw.ellipse([(254,254), (258,258)], ROUGE)
draw.text((190,260), '(CIRCONF/2,CIRCONF/2)', ROUGE)
draw.ellipse([(508,508), (518,518)], ROUGE)
draw.text((400,500), '(CIRCONF,CIRCONF)', ROUGE)

display(repere)
#repere.save('img/repere_wmts.jpg')

Constante:

In [ ]:
f"Circonférence de la terre: {CIRCONF/1000:.3f} km"

- Coordonnées WMTS (projection Web Mercator avec changement de repère):

$\begin{cases}x_{wmts}=\dfrac{CIRCONF}{2} + R_{Terre}\cdot longitude_{[rad]}\\y_{wmts} = \dfrac{CIRCONF}{2} - R_{Terre}\cdot \ln\left( \tan\left( \dfrac{latitude_{[rad]}}{2}+\dfrac{\pi}{4} \right) \right)\end{cases}$

In [ ]:
def wmts(latitude, longitude):
    '''
    Calcule les coordonnées WMTS (en mètres)
    d'une géolocalisation donnée (latitude et longitude en degrés)
    '''
    return (CIRCONF/2 + R_TERRE * math.radians(longitude),
        CIRCONF/2 - R_TERRE * math.log(math.tan(math.radians(latitude)/2 + math.pi/4)) )

In [ ]:
# Lycée Vizille, ..., ...
wmts(45.068116, 5.768002), wmts(0, 0), wmts(0, -180)

## Niveaux de résolution et tuiles WMTS

À chaque niveau de résolution *n*, l'image correspondante de la Terre est subdivisée en $2^n\times 2^n$ tuiles (niveau 0: 1 tuile, niveau 1: 4 tuiles, niveau 2: 16 tuiles&hellip;). Chaque tuile est identifiée par ses indices de ligne et de colonne.

In [ ]:
niveau2 = Image.open('img/terre.jpg')
draw = ImageDraw.Draw(niveau2)

draw.rectangle([(0,0), (511,511)], None, BLEU, 1)
for i in range(4):
    for j in range(4):
        draw.rectangle([(128*j,128*i), (128*j+128,128*i+128)], None, BLEU, 1)
        draw.text((128*j+45,128*i+60), f"n2-({i},{j})", BLEU)

display(niveau2)
#niveau2.save('img/wmts_n2.jpg')

Pour un niveau donnée, chaque tuile est un carré dont les côtés couvrent une distance (en mètre):

In [ ]:
def etendue_tuile(niveau):
    '''Calcule l'étendue d'une tuile en mètres WMTS 
    (différents de mètres réels sauf au niveau de l'équateur)
    '''
    return (CIRCONF / (2**niveau))
etendue_tuile(15)

Les indices de ligne et de colonne de la tuile de niveau *n* qui contient une géolocalisation donnée se calculent de la manière suivante:
$\begin{cases}l = \dfrac{y_{wmts}}{distance\_tuile(n)}\\c = \dfrac{x_{wmts}}{distance\_tuile(n)}\end{cases}$

In [ ]:
def tuile_info(latitude, longitude, niveau=15):
    '''
    Calcule les indices de ligne et de colonne de la tuile
    contenant la géolocalisation ainsi que les coordonnées
    x et y (en pixel) de cette géolocalisation dans cette
    tuile.
    '''
    cote = etendue_tuile(niveau)
    coord = wmts(latitude, longitude)
    ligne, colonne = int(coord[1]/cote), int(coord[0]/cote)
    x = int(math.modf(coord[0]/cote)[0] * 256)
    y = int(math.modf(coord[1]/cote)[0] * 256)
    return ligne, colonne, x, y

In [ ]:
# Lycée Vizille, ...
tuile_info(45.068116, 5.768002, 2), tuile_info(45.068116, -5.768002, 2)

## Géoservices

https://geoservices.ign.fr/

- [clé essentiels](https://geoservices.ign.fr/services-web-essentiels): accès aux principaux services web, gratuitement et sans inscription.
- [Scans IGN](https://geoservices.ign.fr/services-web-issus-des-scans-ign): SCAN 25®, SCAN 100®&hellip;, gratuitement (jusqu'à un certain seuil) et avec inscription (clé personnelle)
- &hellip;

In [ ]:
def afficher_capacites():
    url = f"https://wxs.ign.fr/essentiels/geoportail/wmts?SERVICE=WMTS&REQUEST=GetCapabilities"
    print(url)

    reponse=requests.get(url)

    x = dom.parseString(reponse.text)
    print(x.toprettyxml(indent='  ', newl='\n'))

In [ ]:
afficher_capacites()

### Cartes IGN

- Récupérer une tuile connaissant ses indices:

In [ ]:
def get_tuile_ign(l, c, niveau=15, ortho=False):
    url = (f"https://wxs.ign.fr/essentiels/geoportail/wmts/?"
           "SERVICE=WMTS&VERSION=1.0.0&REQUEST=GetTile"
           "&TILEMATRIXSET=PM" 
           f"&TILEMATRIX={niveau}&TILEROW={l}&TILECOL={c}"
           f"&STYLE=normal")
    if ortho:
        url = url + "&LAYER=ORTHOIMAGERY.ORTHOPHOTOS&FORMAT=image/jpeg"
    else:
        url = url + "&LAYER=GEOGRAPHICALGRIDSYSTEMS.PLANIGNV2&FORMAT=image/png"
    
    return Image.open(requests.get(url, stream=True).raw)

Exemples:

In [ ]:
# La tuile orthographique du niveau 0
terre_n0 = get_tuile_ign(0,0,0)
draw = ImageDraw.Draw(terre_n0)
draw.rectangle([(0,0), (255,255)], None, GRIS)
display(terre_n0)
#terre_n0.save('img/terre_n0.jpg')

In [ ]:
# Les 4 tuiles orthographiques du niveau 1
terre_n1 = Image.new('RGB', (2*256, 2*256))
for i in range(2):
    for j in range(2):
        t = get_tuile_ign(i,j,1)
        terre_n1.paste(t, (256*j, 256*i))

draw = ImageDraw.Draw(terre_n1)
for i in range(2):
    for j in range(2):
        draw.rectangle([(j*256,i*256), (j*256+255,i*256+255)], None, GRIS)

display(terre_n1)
#terre_n1.save('img/terre_n1.jpg')

In [ ]:
# Les 16 tuiles orthographiques du niveau 2
terre_n2 = Image.new('RGB', (4*256, 4*256), BLANC)
for i in range(4):
    for j in range(4):
        print(i,j)
        t = get_tuile_ign(i,j,2)
        terre_n2.paste(t, (256*j, 256*i))

draw = ImageDraw.Draw(terre_n2)
for i in range(4):
    for j in range(4):
        draw.rectangle([(j*256,i*256), (j*256+255,i*256+255)], None, GRIS)
display(terre_n2)
#terre_n2.save('img/terre_n2.jpg')

- Récupérer une tuile contenant une géolocalisation spécifique:

In [ ]:
def get_geo_tuile_ign(latitude, longitude, niveau=15, ortho=False, dispLoc=False):
    (l,c, x, y) = tuile_info(latitude, longitude, niveau)
    t = get_tuile_ign(l, c, niveau, ortho)
    if dispLoc:
        draw = ImageDraw.Draw(t)
        draw.ellipse([(x-4, y-4), (x+4, y+4)], (255,0,0))
    return t

Exemple:

In [ ]:
# Coordonnées de la tour Eiffel
latitude, longitude = 48.8584, 2.2945
# Coordonnées de Vizille
latitude, longitude = 45.068116, 5.768002

t = get_geo_tuile_ign(latitude, longitude, 15, ortho=False, dispLoc=True)
display(t)

In [ ]:
from ipywidgets import interact, fixed, widgets
interact(get_geo_tuile_ign, 
         latitude=fixed(45.068116), longitude=fixed(5.768002),
         niveau=widgets.IntSlider(value=15,min=0,max=18, continuous_update=False), ortho=False, dispLoc=True)

### Géocodage

Ce service permet d'obtenir la géolocalisation d'une adresse.

(https://geoservices.ign.fr/documentation/geoservices/geocodage.html)

In [ ]:
def localiser(adresse):
    content = ( "<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n"
                "<XLS\n"
                "  xmlns:gml=\"http://www.opengis.net/gml\"\n"
                "  xmlns=\"http://www.opengis.net/xls\"\n"
                "  xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" version=\"1.2\"\n"
                "  xsi:schemaLocation=\"http://www.opengis.net/xls http://schemas.opengis.net/ols/1.2/olsAll.xsd\">\n"
                "  <RequestHeader srsName=\"epsg:4326\"/>\n"
                "  <Request maximumResponses=\"1\" methodName=\"GeocodeRequest\" requestID=\"uid42\" version=\"1.2\">\n"
                "  <GeocodeRequest returnFreeForm=\"false\">\n"
                "    <Address countryCode=\"StreetAddress\">\n"
                f"      <freeFormAddress>{adresse}</freeFormAddress>\n"
                "    </Address>\n"
                "  </GeocodeRequest>\n"
                "  </Request>\n"
                "</XLS>\n")
    url = f"https://wxs.ign.fr/essentiels/geoportail/ols?"
    reponse = requests.post(url, headers={'Content-Type': 'text/xml'}, data=content)
    doc = dom.parseString(reponse.text)
    coord = doc.getElementsByTagName("gml:pos")[0].firstChild.toprettyxml().rstrip().split(' ')
    
    return float(coord[0]), float(coord[1]) # latitude, longitude

In [ ]:
latitude, longitude = localiser('960 avenue Aristide Briand 38220 Vizille')
t = get_geo_tuile_ign(latitude, longitude, 15, dispLoc=True)
display(t)